In [ ]:
# Import necessary modules
import json
from datetime import datetime

# JSON Storage Handler
class JSONStorageHandler:
  """
  Handles reading and writing JSON data related to products and sales.

  Attributes:
  file_name: The name of the JSON file for data storage.
  data: Dictionary containing 'products' and 'sales' data.

  Methods:
  load_data(): Loads data from the JSON file or initializes an empty structure if the file is invalid.
  save_data(): Saves the current data to the JSON file.
  add_product(name, product): Adds or updates a product in the inventory.
  update_product(name, quantity_removed): Updates the quantity of a product after a sale.
  record_sale(sale): Records the details of a sale transaction.
  """
  def __init__(self, file_name="biomarket.json"):
    """
    Initializes the JSON storage handler.

    Parameters:
    file_name (str): The name of the file where the data will be stored ('biomarket.json').
    """
    self.file_name = file_name
    self.data = {"products": {}, "sales": []}
    self.load_data()

  def load_data(self):
    """
    Loads data from the JSON file or initializes a new file if invalid or non-existent.
    """
    try:
      with open(self.file_name, 'r') as file:
        self.data = json.load(file)
      if "products" not in self.data or "sales" not in self.data:
        raise ValueError("File JSON non valido.")
    except (FileNotFoundError, ValueError, json.JSONDecodeError):
      self.data = {"products": {}, "sales": []}
      self.save_data()

  def save_data(self):
    """
    Saves the current data to the JSON file.
    """
    with open(self.file_name, 'w') as file:
      json.dump(self.data, file, indent=4)

  def add_product(self, name, product):
    """
    Adds a product to the inventory or updates the quantity if the product exists.

    Parameters:
    name (str): The name of the product.
    product (dict): Dictionary containing details about the product (quantity, purchase price, sale price).
    """
    if name in self.data["products"]:
      self.data["products"][name]["quantity"] += product["quantity"]
    else:
      self.data["products"][name] = product
    self.save_data()

  def update_product(self, name, quantity_removed):
    """
    Updates the quantity of a product after a sale.

    Parameters:
    name (str): The name of the product.
    quantity_removed (int): The quantity of the product sold and removed from inventory.
    """
    if name in self.data["products"]:
      self.data["products"][name]["quantity"] -= quantity_removed
    self.save_data()

  def record_sale(self, sale):
    """
    Records a sale transaction.

    Parameters:
    sale (dict): A dictionary containing the sale details (date, products sold, total).
    """
    self.data["sales"].append(sale)
    self.save_data()

# BioMarket main class
class BioMarket:
  """
  Handles main operations of the organic store, such as managing products and sales.

  Attributes:
  storage (JSONStorageHandler): An instance of the storage handler to manage JSON data.

  Methods:
  input_integer(message): Prompts the user to input a positive integer.
  input_float(message): Prompts the user to input a positive float or leave it empty.
  input_choice(message, options): Prompts the user to select one option from a list.
  add_product(name, quantity, purchase_price=None, sale_price=None): Adds a product to the inventory or updates its quantity.
  update_product_prices(name, new_purchase_price=None, new_sale_price=None): Updates the prices of a product.
  list_products(): Lists all available products in the inventory.
  record_sale(): Records the details of a sale.
  calculate_profits(): Calculates both gross and net profits from the sales.
  show_help(): Displays available commands for the user.
  """
  def __init__(self, storage_handler):
    """
    Initializes the BioMarket class with a storage handler.

    Parameters:
    storage_handler (JSONStorageHandler): Instance of JSONStorageHandler to manage the data storage.
    """
    self.storage = storage_handler

  def input_integer(self, message):
    """
    Prompts the user to enter a positive integer.

    Parameters:
    message (str): The prompt message shown to the user.

    Returns:
    int: The positive integer input by the user.
    """
    while True:
      try:
        value = int(input(message))
        if value < 0:
          print("Errore: il numero deve essere positivo.")
          continue
        return value
      except ValueError:
        print("Errore: inserisci un numero intero valido.")

  def input_float(self, message):
    """
    Prompts the user to input a positive float or leave it blank.

    Parameters:
    message (str): The prompt message shown to the user.

    Returns:
    float: The positive float input by the user or None if left blank.
    """
    while True:
      try:
        user_input = input(message)
        if not user_input.strip():
          return None
        value = float(user_input)
        if value < 0:
          print("Errore: il numero deve essere positivo.")
          continue
        return value
      except ValueError:
        print("Errore: inserisci un numero valido.")

  def input_choice(self, message, options):
    """
    Prompts the user to select a choice from a list of valid options.

    Parameters:
    message (str): The prompt message shown to the user.
    options (list): List of valid choices.

    Returns:
    str: The choice selected by the user.
    """
    while True:
      choice = input(message).strip().lower()
      if choice in options:
        return choice
      print(f"Errore: scelta una delle opzioni valide: {', '.join(options)}.")

  def add_product(self, name, quantity, purchase_price=None, sale_price=None):
    """
    Adds a product to the inventory or updates the quantity if it already exists.

    Parameters:
    name (str): The product name.
    quantity (int): The number of units of the product to add.
    purchase_price (float, optional): The purchase price of the product (default is None).
    sale_price (float, optional): The sale price of the product (default is None).
    """
    while quantity <= 0:
      print("Errore: la quantità deve essere maggiore di zero.")
      quantity = self.input_integer("Quantità: ")

    if name in self.storage.data["products"]:
      self.storage.data["products"][name]["quantity"] += quantity
      print(f"AGGIORNATO: {quantity} X {name}, quantità incrementata.")
    else:
      if purchase_price is None or purchase_price <= 0:
        print("Errore: il prezzo di acquisto deve essere maggiore di zero.")
        return
      if sale_price is None or sale_price <= 0:
        print("Errore: il prezzo di vendita deve essere maggiore di zero.")
        return
      product = {
        "quantity": quantity,
        "purchase_price": purchase_price,
        "sale_price": sale_price
      }
      self.storage.data["products"][name] = product
      print(f"AGGIUNTO: {quantity} X {name}")
    self.storage.save_data()

  def update_product_prices(self, name, new_purchase_price=None, new_sale_price=None):
    """
    Updates the purchase and/or sale prices of a product.

    Parameters:
    name (str): The product name.
    new_purchase_price (float, optional): The new purchase price for the product (default is None).
    new_sale_price (float, optional): The new sale price for the product (default is None).
    """
    if name in self.storage.data["products"]:
      if new_purchase_price is not None:
        while new_purchase_price is None or new_purchase_price <= 0:
          print("Errore: il prezzo di acquisto deve essere maggiore di zero.")
          new_purchase_price = self.input_float("Nuovo prezzo di acquisto: ")

        self.storage.data["products"][name]["purchase_price"] = new_purchase_price
        print(f"Prezzo di acquisto aggiornato per {name}: €{new_purchase_price:.2f}.")

      if new_sale_price is not None:
        while new_sale_price is None or new_sale_price <= 0:
          print("Errore: il prezzo di vendita deve essere maggiore di zero.")
          new_sale_price = self.input_float("Nuovo prezzo di vendita: ")

        self.storage.data["products"][name]["sale_price"] = new_sale_price
        print(f"Prezzo di vendita aggiornato per {name}: €{new_sale_price:.2f}.")

      self.storage.save_data()
    else:
      print(f"Errore: il prodotto '{name}' non è registrato in magazzino.")

  def list_products(self):
    """
    Lists all products in the inventory.

    Displays the product name, quantity available, and sale price.
    """
    print("PRODOTTO\tQUANTITA'\tPREZZO")
    for name, details in self.storage.data["products"].items():
      if details['quantity'] > 0:
        print(f"{name}\t{details['quantity']}\t€{details['sale_price']}")

  def record_sale(self):
    """
    Records a sale transaction.

    It prompts the user to input the product names and quantities, computes the total sale amount,
    updates inventory, and logs the sale in the system.
    """
    total_sale = 0
    current_sale = []

    while True:
      name = input("Nome del prodotto: ").strip()
      if name not in self.storage.data["products"]:
        print(f"Errore: il prodotto '{name}' non è disponibile in magazzino.")
        continue_choice = self.input_choice("Vuoi provare con un altro prodotto? (si/no): ", ["si", "no"])
        if continue_choice == "no":
          return
        continue

      product = self.storage.data["products"][name]
      if "sale_price" not in product or product["sale_price"] is None:
        print(f"Errore: il prodotto '{name}' non ha un prezzo di vendita.")
        return

      available_quantity = product["quantity"]
      quantity = self.input_integer(f"Quantità (disponibile: {available_quantity}): ")
      if quantity > available_quantity or quantity <= 0:
        print(f"Errore: quantità non valida.")
        continue

      self.storage.update_product(name, quantity)
      total_price = quantity * product["sale_price"]
      total_sale += total_price
      current_sale.append({
          "name": name,
          "quantity": quantity,
          "total_price": total_price,
          "unit_price": product["sale_price"]
      })

      another = self.input_choice("Aggiungere un altro prodotto? (si/no): ", ["si", "no"])
      if another == "no":
        break

    self.storage.record_sale({
        "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "products": current_sale,
        "total": total_sale
    })

    print("VENDITA REGISTRATA")
    for item in current_sale:
      print(f"- {item['quantity']} X {item['name']}: €{item['unit_price']:.2f}")

    print(f"Totale: €{total_sale:.2f}")

  def calculate_profits(self):
    """
    Calculates total profits: gross and net.

    It calculates gross profit (total sales amount) and net profit
    (after subtracting product purchase costs) from all sales recorded.
    """
    gross_profit = 0
    net_profit = 0

    for sale in self.storage.data["sales"]:
      gross_profit += sale["total"]
      for product in sale["products"]:
        name = product["name"]
        quantity = product["quantity"]
        if "unit_price" not in product:
          product_price = self.storage.data["products"].get(name, {}).get("sale_price", 0)
        if "total_price" not in product:
          product["total_price"] = quantity * product["unit_price"]
        total_price = product["total_price"]
        purchase_price = self.storage.data["products"].get(name, {}).get("purchase_price", 0)
        net_profit += total_price - (quantity * purchase_price)

    print(f"Profitto: lordo=€{gross_profit:.2f} netto=€{net_profit:.2f}")

  def show_help(self):
    """
    Displays a list of available commands for the user.
    """
    print("I comandi disponibili sono i seguenti:\n"
          "aggiungi: aggiungi un prodotto al magazzino\n"
          "aggiorna: aggiorna i prezzi di un prodotto\n"
          "elenca: elenca i prodotti in magazzino\n"
          "vendita: registra una vendita effettuata\n"
          "profitti: mostra i profitti totali\n"
          "aiuto: mostra i possibili comandi\n"
          "chiudi: esci dal programma")

def main():
  """
  Main entry point of the program.
  The user interacts with the BioMarket management system by using commands.
  """
  storage = JSONStorageHandler()
  management_system = BioMarket(storage)

  while True:
    command = input("Inserisci un comando: ").strip().lower()
    if command == "chiudi":
      print("Bye bye")
      break
    elif command == "aggiungi":
      name = input("Nome del prodotto: ").strip()
      quantity = management_system.input_integer("Quantità: ")
      while quantity <= 0:
        print("Errore: la quantità deve essere maggiore di zero.")
        quantity = management_system.input_integer("Quantità: ")
      if name in storage.data["products"]:
        management_system.add_product(name, quantity)
      else:
        purchase_price = management_system.input_float("Prezzo di acquisto: ")
        while purchase_price is None or purchase_price <= 0:
          print("Errore: il prezzo di acquisto deve essere maggiore di zero.")
          purchase_price = management_system.input_float("Prezzo di acquisto: ")

        sale_price = management_system.input_float("Prezzo di vendita: ")
        while sale_price is None or sale_price <= 0:
          print("Errore: il prezzo di vendita deve essere maggiore di zero.")
          sale_price = management_system.input_float("Prezzo di vendita: ")
        management_system.add_product(name, quantity, purchase_price, sale_price)
    elif command == "aggiorna":
      name = input("Nome del prodotto da aggiornare: ").strip()
      new_purchase_price = management_system.input_float("Nuovo prezzo di acquisto (lascia vuoto per non aggiornare): ")
      new_sale_price = management_system.input_float("Nuovo prezzo di vendita (lascia vuoto per non aggiornare): ")
      management_system.update_product_prices(name, new_purchase_price, new_sale_price)
    elif command == "elenca":
      management_system.list_products()
    elif command == "vendita":
      management_system.record_sale()
    elif command == "profitti":
      management_system.calculate_profits()
    elif command == "aiuto":
      management_system.show_help()
    else:
      print("Comando non valido")
      management_system.show_help()

# Start the program
if __name__ == "__main__":
  main()

Inserisci un comando: aggiungi
Nome del prodotto: latte di soia
Quantità: -5
Errore: il numero deve essere positivo.
Quantità: 0
Errore: la quantità deve essere maggiore di zero.
Quantità: 20
Prezzo di acquisto: -0.80
Errore: il numero deve essere positivo.
Prezzo di acquisto: 0
Errore: il prezzo di acquisto deve essere maggiore di zero.
Prezzo di acquisto: 0.80
Prezzo di vendita: -1.40
Errore: il numero deve essere positivo.
Prezzo di vendita: 0
Errore: il prezzo di vendita deve essere maggiore di zero.
Prezzo di vendita: 1.40
AGGIUNTO: 20 X latte di soia
Inserisci un comando: elenca
PRODOTTO	QUANTITA'	PREZZO
latte di soia	20	€1.4
Inserisci un comando: chiudi
Bye bye
